In [2]:
import os
from tkinter import *
#from mttkinter import mtTkinter
from tkinter import filedialog
import pandas as pd
pd.options.mode.chained_assignment = None
import glob
from natsort import natsorted
from datetime import date

In [3]:
todays_date = date.today()
todays_date.strftime('20%y%m%d')

'20211020'

In [4]:
root = Tk()
root.title('Axiom LIMS File Handler')
root.geometry('800x600')


projectname = StringVar(root)
platenum = StringVar(root)

projectname.set('Select Project')

dropdown = ['IntellxxDNA_MethylGenetics', 'Merck', 'Genetica']
projectmenu = OptionMenu(root, projectname, *dropdown)
projectmenu.place(x=350,y=150)

entlabel = Label(root, text='Enter Plate Number:')
entlabel.place(x=230,y=200)

platenum = Entry(root)
platenum.place(x=350,y=200)


filepath = os.path.join(r'R:\Brookslab Technicians ISI\Project Folders')

class Q_FileGen:
    def readxcelConSL(self):
        for filename in glob.glob(os.path.join(filepath, projectname.get(), 'Manifests', '*.xlsx')):
            filename.replace('p','').replace('P','')
            remolist = ['Axiom_Pos','NTC']
            if projectname.get() in filename and platenum.get() in filename:
                if projectname.get() == 'IntellxxDNA_MethylGenetics':

                    Barcode_df=pd.read_excel(r'R:\Brookslab Technicians ISI\Project Folders\IntellxxDNA_MethylGenetics\IntelXX_Barcode_Tracking_Sheet.xlsx', 
                                             index_col = 'Plate Number')
                    Barcode_df.columns = Barcode_df.columns.str.rstrip().str.lstrip()
                    TubeID = Barcode_df.loc[['Intelxx P'+platenum.get()],['Beckman']].values.item()
                    
                    working_volumn = '20'
                    volumn_unit = 'ul'
                    
                    Manifest_df = pd.read_excel(os.path.join(filepath,'Manifests', filename))
                    Manifest_df = Manifest_df[~Manifest_df['Derivative_Tube_Barcode'].str.contains('|'.join(remolist), na=False)]
                    #natsorted(Manifest_df['Well'], key=lambda x: x.lower())
                    
                    SL_df = Manifest_df[['Derivative_Tube_Barcode', 'Plate_Barcode', 'Well']]
                    SL_df.columns = ['Inventory_Code','Box_Number','Position']
                    #SL_df.to_csv(os.path.join(filepath,projectname.get(),'SL Files', projectname.get()+'_P'+platenum.get()+'_SL_Files.csv'), index=False)
                    #os.startfile(os.path.join(filepath,projectname.get(),'SL Files', projectname.get()+'_P'+platenum.get()+'_SL_Files.csv'))
                    #SL_df[['Inventory_Code']].to_csv(os.path.join(filepath,projectname.get(),'IntelXX Inventory codes', 'Intelxx_P'+platenum.get()+'_Inv_codes.csv'),header=0, index=False)
                    #os.startfile(os.path.join(filepath,projectname.get(),'IntelXX Inventory codes', 'Intelxx_P'+platenum.get()+'_Inv_codes.csv'))
                    print(SL_df)
                    
                    consume_df = Manifest_df[['Derivative_Tube_Barcode']]
                    consume_df['Consume_Amt'] = working_volumn
                    consume_df['Consume_Unit'] = volumn_unit
                    consume_df['Consume_Reason'] = 'Genotyping Processing'
                    consume_df.columns = ['Consume_Inventory_Code', 'Consume_Amt', 'Consume_Unit', 'Consume_Reason']
                    #consume_df.to_csv(os.path.join(filepath,projectname.get(),'Consume Files', projectname.get()+'_P'+platenum.get()+'_Consume_Files.csv'), index=False)
                    #os.startfile(os.path.join(filepath,projectname.get(),'Consume Files', projectname.get()+'_P'+platenum.get()+'_Consume_Files.csv'))
                   
                    
                    cols = SL_df.columns.tolist()
                    cols_order = [1,2,0]
                    cols = [cols[i] for i in cols_order]
                    self.Micronic_to_Beckman_df = SL_df[cols]
                    self.Micronic_to_Beckman_df['Destination Box #'] = TubeID
                    self.Micronic_to_Beckman_df['Destination Position'] = self.Micronic_to_Beckman_df[['Position']]
                    self.Micronic_to_Beckman_df.columns = ['Source Box #','Source Position','Tube ID','Destination Box #','Destination Position']
                    
                    
                    print(self.Micronic_to_Beckman_df)
        
        
                #else:
                    #labelN = Label(root, text='File Not Found')
                    #labelN.place(x=350,y=350)       

    def file_browse(self):
        root.filename = filedialog.askopenfilename(initialdir=r'‪C:\Users\swang\Desktop', title = 'Select a file', filetypes = (('All Files','*.*'), ('Excel','*.xlsx'), ('CSV','*.csv')))
        self.query_df = pd.read_excel(root.filename)
        self.query_df.columns.str.rstrip().str.lstrip()
        file_name = os.path.basename(root.filename)
        file_label = Label(root, text=file_name).place(x=350,y=400)


    def file_gen(self):
        #self.query_df.set_index('SERVICENAME')
        matcode_df = self.query_df.loc[self.query_df['SERVICENAME'] == 'TEST GENO A']
        #matcode_df = matcode_df.loc[['Next_Step'],['Add Reagent']]
        matcode_df = matcode_df[['INVENTORY_CODE','MATCODE']]
        matcode_df.columns = ['Tube ID','MATCODE']
        matcode_df = matcode_df.drop_duplicates('Tube ID')
        matcode_df = matcode_df.reset_index()
        del matcode_df['index']
        mat_df_all = self.Micronic_to_Beckman_df.merge(matcode_df,on='Tube ID', how='left')
        
        unique_mat =  mat_df_all['MATCODE'].unique()
        
        uniquelength = list(range(len(unique_mat)))
        for i in uniquelength:
            globals()['matdf' + str(i)] = mat_df_all.loc[mat_df_all['MATCODE'] == unique_mat[i]]
            #matdf2 = mat_df_all.loc[mat_df_all['MATCODE'] == unique_mat[i]]
        
        print(matcode_df)
        print(mat_df_all)
        print(uniquelength)
        
        


        
q = Q_FileGen()

GenFile_Button = Button(root, text='Generate Consume/SL/Inv File', command=q.readxcelConSL).place(x=350,y=250)

browse_button = Button(root, text = 'Browse Query File', command=q.file_browse).place(x=300,y=350)

gen_button = Button(root, text='Generate', command=q.file_gen).place(x=400,y=350)    

root.mainloop()


   Inventory_Code  Box_Number Position
0      8049865270  7000460515      A01
1      8049896284  7000460515      B01
2      8049896296  7000460515      C01
3      8049896308  7000460515      D01
4      8049896320  7000460515      E01
..            ...         ...      ...
89     8049896362  7000460515      B12
90     8049896374  7000460515      C12
91     8049896291  7000460515      D12
92     8049896303  7000460515      E12
93     8049896315  7000460515      F12

[94 rows x 3 columns]
    Source Box # Source Position     Tube ID Destination Box #  \
0     7000460515             A01  8049865270         BC0003581   
1     7000460515             B01  8049896284         BC0003581   
2     7000460515             C01  8049896296         BC0003581   
3     7000460515             D01  8049896308         BC0003581   
4     7000460515             E01  8049896320         BC0003581   
..           ...             ...         ...               ...   
89    7000460515             B12  8049896362   